In [1]:
#!pip install pyvisa

In [1]:
import pyvisa
import numpy as np
import time
import matplotlib.pyplot as plt
import os
import csv
import tkinter as tk
from tkinter import messagebox
import threading
from threading import Event
%matplotlib qt

INITIALIZE

In [11]:
rm = pyvisa.ResourceManager()
# Keithley 2400
keith = rm.open_resource('GPIB0::23::INSTR')
keith.timeout = 100000
keith.write('*RST')

  
# picoamp meter
pico = rm.open_resource('GPIB0::22::INSTR')
pico.timeout = 10000
pico.write('*RST')

6

In [3]:
# picoamp meter
rm = pyvisa.ResourceManager()
m2657A = rm.open_resource('TCPIP0::132.181.53.204::inst0::INSTR')
m2657A.timeout = 10000
m2657A.write('*RST')

6

In [21]:
# m2657A.write("reset()")
# device = m2657.read()
# print(device)
# print(m2657.read())
m2657A.write(f"beeper.beep(2,100)")

20

In [27]:
sweepFilePath = "C:\\Users\\gtd19\\Semsem\\bGa2O3\\KeithleyControl\\TSP_2657A\\2657A_DiodeRL\\DiodeRL.tsp"

m2657A.write("loadscript simpleSweep")
with open(sweepFilePath) as fp:
    for line in fp:
        m2657A.write(line)
m2657A.write("endscript")
m2657A.write("simpleSweep.run()")
fp.close()

In [4]:
def update_plot(fig, ax, line, lstVoltage, lstAbsCurrent):
    line.set_data(lstVoltage, lstAbsCurrent)
    ax.relim()
    ax.autoscale_view()
    fig.canvas.draw()
    fig.canvas.flush_events()

In [5]:
def fncMeasureOneKL(arrVMeasurement, \
                  Sample = 'Test', sweepNum = 1,\
                  folderPath='230530\\Dev02\\Test',\
                  askToStart=False):
    Vstart, Vstop, Vstep = arrVMeasurement
    fileName = "Sweep"+str(sweepNum)+Sample+'Star'+str(Vstart).replace("-","M")+'_Stop'+str(Vstop).replace("-","M")+'_Step'+str(Vstep).replace("-", "M")+'.csv'
    filePath = os.path.join(folderPath, fileName)
    print(filePath)
    stepNum = np.floor(abs((Vstart - Vstop)/Vstep)).astype(int) + 1
    vRange = "AUTO"
    iCompliance = 1
    iNPLC = 1
    delay = 0.1
    keith.write('*RST')
    keith.write(':ROUT:TERM REAR')
    keith.write(':SOUR:FUNC VOLT')
    keith.write(':SENS:FUNC "CURR:DC"')
    keith.write(":SOUR:VOLT:STAR %0.2f" % Vstart)
    keith.write(":SOUR:VOLT:STOP %0.2f" % Vstop)
    keith.write(":SOUR:VOLT:STEP %0.3f" % Vstep)
    vRange = ':SOUR:SWE:RANG ' + vRange
    keith.write(vRange)
    iCompliance = ':SENSE:CURR:PROT '+ str(iCompliance)
    keith.write(iCompliance)
    keith.write(":SOUR:VOLT:MODE SWE")
    keith.write(":SENSE:CURR:PROT 1" )
    keith.write(':SENSE:CURR:NPLC %f' %iNPLC )
    keith.write(':SOUR:SWE:SPAC LIN')
    keith.write("TRAC:CLE")
    keith.write('TRAC:POIN %f' %stepNum)
    keith.write(':FORM:ELEM:SENS VOLT,CURR')
    keith.write(':TRIG:COUN %f' % stepNum)
    keith.write(':SOUR:SWE:POIN %f' %stepNum)
    keith.write(':SOUR:DEL %0.2f' % delay)
    keith.write(':OUTP ON')
    keith.write(":INIT")
    keith.write('*WAI')
    keith.write(':OUTP OFF')

    time.sleep(1)

    data = keith.query_ascii_values("FETC?")
    #print(data)
    # Read data from buffer of Keithley 2400 and write to the file
    lstCurrent = data[1::2] # get all odd elements, which are currents

    lstVoltage = data[0::2] # get all even elements, which are voltages

    # if Vstart > Vstop:
    #     Vstep = -abs(Vstep)
    # else:
    #     Vstep = abs(Vstep)
    lstAbsCurrent = np.abs(lstCurrent)
    lstCalVoltage = [(Vstart + i*Vstep) for i in range(stepNum)]
    dataToWrite = [['vRange', vRange]]
    dataToWrite.append(['iCompliance', iCompliance])
    #dataToWrite.append(['iRange', iRange[16:]])
    dataToWrite.append(['iNPLC', iNPLC])
    dataToWrite.append(['data Number', stepNum])
    dataToWrite = [['V', 'I', 'V_cal','abs_I']]
    for i in range(len(lstCurrent)):
        dataToWrite.append([lstVoltage[i], lstCurrent[i], lstCalVoltage[i] , lstAbsCurrent[i]])
    try:
        with open(filePath, 'w', newline = '') as file:
            writer = csv.writer(file)
            writer.writerows(dataToWrite)
            file.close()
    except IOError:
        print("Error: Could not open file for writing")

    fig, ax = plt.subplots()
    line, = ax.plot(lstCalVoltage, lstAbsCurrent, 'o-')
    ax.set_xlabel('Voltage (V)')
    #plt.xticks(fontsize = 18)
    ax.set_ylabel('Current (A)')
    ax.set_yscale('log')
    #plt.yticks(fontsize = 18)
    # Function to update plot
    plt.show()
    plt.savefig(os.path.splitext(filePath)[0] + '.png')


In [6]:
def fncMeasureOnePico(arrVMeasurement, \
                  Sample = 'Test', sweepNum = 1,\
                  folderPath='230530\\Dev02\\Test',\
                  askToStart=False):
    global running
    # Create the main application window
    
    # Create a button to exit the loop
    exit_button = tk.Button(root, text="Exit Measurements", command=exitLoop)
    exit_button.pack()
    Vstart, Vstop, Vstep = arrVMeasurement
    fileName = "Sweep"+str(sweepNum)+Sample+'Star'+str(Vstart).replace("-","M")+'_Stop'+str(Vstop).replace("-","M")+'_Step'+str(Vstep).replace("-", "M")+'.csv'
    filePath = os.path.join(folderPath, fileName)
    print(filePath)
    stepNum = np.floor(abs((Vstart - Vstop)/Vstep)).astype(int) + 1
    vRange = 500
    vILIM = 2e-3
    iRange = '2e-9'
    iNPLC = 1
    #vILIM = 'SOUR:VOLT:ILIM' + vILIM
    iRange = ':SENSE:CURR:RANG ' + iRange
    breakDownVolt = 505
    breakDownCurr = 2.5e-3
    pico.write('*RST')
    pico.write(iRange)
    pico.write("SOUR:VOLT:RANG %f" % vRange)
    pico.write("SOUR:VOLT:ILIM %f" %vILIM)
    pico.write(":SENSE:CURR:NPLC %f" %iNPLC )
    lstCurrent = []
    lstVoltage = []
    lstAbsCurrent = []
    data = [['Volt', 'Curr'],['V', 'A']]
    fig, ax = plt.subplots()
    line, = ax.plot([], [], 'o-')
    ax.set_xlabel('Voltage (V)')
    ax.set_ylabel('Absolute Current (A)')
    ax.set_yscale('log')
    
    
#     messStartMeasurement = 'File Paht: ' + filePath + '.\n' + " Do you want to start the measurement?" 
#     # Display a message box and ask for the operator's choice
#     choice = messagebox.askquestion("Measurement", messStartMeasurement)
    
#     if choice == 'yes':
    plt.show()
    time.sleep(1)
    pico.write(':SOUR:VOLT:STAT ON')
    curExceedLimitMeasNum = 0 # number of times the current exceeds the current limit - error
    afterBreakDownMeasNum = 10 # number of additional measurements after the current reaches the current limit at the flat region
    curLimitError = 0.0003
    curLimitLowerBound = breakDownCurr - curLimitError
    # Measurement loop
    for i in range(stepNum):
#         if not running:
#             break
        
        lstVoltage.append(Vstart + i*Vstep)
        if np.absolute(Vstart + i*Vstep) > breakDownVolt:
            break
        pico.write(':INIT:IMM')
        #strVol = '{0:.2f}'.format(i*0.2)
        strVol = ':SOUR:VOLT {0:.2f}'.format(Vstart + i*Vstep)
        pico.write(strVol)
        results = pico.query(':MEASure:CURRent?')
        #print(results)
        current = results.split(',')[0][:-1]
        #voltage = l
        lstCurrent.append(float(np.squeeze(current)))
        lstAbsCurrent.append(np.abs(float(np.squeeze(current))))
        update_plot(fig, ax, line,lstVoltage, lstAbsCurrent)
        if (abs(float(np.squeeze(current))) >  breakDownCurr): #or (stop_flag == True)
            break
        if (abs(float(np.squeeze(current))) >  curLimitLowerBound):
            curExceedLimitMeasNum+=1
        if (curExceedLimitMeasNum >= afterBreakDownMeasNum):
            break
        time.sleep(0.1)
    pico.write(':SOUR:VOLT:STAT OFF')
    dataToWrite = [['vRange', vRange]]
    dataToWrite.append(['vILIMIT', vILIM])
    dataToWrite.append(['iRange', iRange[16:]])
    dataToWrite.append(['iNPLC', iNPLC])
    dataToWrite.append(['data Number', stepNum])
    dataToWrite.append(['V_cal', 'I','abs_I'])
    lstCalVoltage = [(Vstart + i*Vstep) for i in range(len(lstAbsCurrent))]

    for i in range(len(lstCurrent)):
        dataToWrite.append([round(Vstart + i * Vstep, 2), lstCurrent[i], lstAbsCurrent[i]])
    try:
        with open(filePath, 'w', newline = '') as file:
            writer = csv.writer(file)
            writer.writerows(dataToWrite)
            file.close()
    except IOError:
        print("Error: Could not open file for writing")
    plt.show()
    plt.savefig(os.path.splitext(filePath)[0] + '.png')
#     root.destroy()

In [7]:
def fncMeasureOne(arrVMeasurement, \
                  device= 'pico',\
                  Sample = 'Test', sweepNum = 1,\
                  folderPath='230606\\Dev02\\Test',\
                  askToStart=False):
    if (device == 'pico'):
        fncMeasureOnePico(arrVMeasurement, Sample, sweepNum, folderPath, askToStart)
    elif (device == 'keith'):
        fncMeasureOneKL(arrVMeasurement, Sample, sweepNum, folderPath, askToStart)
    else:
        print("Incorrect device name")

In [8]:
running = True
def exitLoop():
    global running
    running = False
root = tk.Tk()
root.title("Measurements")
# Create a button to exit the loop


''

In [9]:
def fncMeasureMultiple(settings =['pico', [[0, 0.5, 0.1]]], \
                  Sample = 'Test',\
                  startNum = 1, \
                  folderPath='230606\\Dev02\\Test',\
                  askToStart=False): 
    global running
    global root
    sweepNum = 0
    # Create the main application window
    dev, arrVMeasurements = settings
    for j in range (len(arrVMeasurements)):
        sweepNum = j + startNum
        
        if not running:
            break
        fncMeasureOne(arrVMeasurements[j], dev , Sample, sweepNum, folderPath)
        time.sleep(5)
    root.destroy()

In [10]:
folderPath = '240805_Fab240727_NiNiO_NegLOnBrassSM'
# Create a new directory
try:
    os.mkdir(folderPath)
except FileExistsError:
    print("Folder already exists")
    
folderPath = folderPath+ '\\Dev07'
# Create a new directory
try:
    os.mkdir(folderPath)
except FileExistsError:
    print("Folder already exists")

folderPath = folderPath+ '\\E02'
# Create a new directory
try:
    os.mkdir(folderPath)
except FileExistsError:
    print("Folder already exists")
sample = folderPath[-3:]

Folder already exists
Folder already exists


In [29]:
#tk.mainloop()

In [57]:
#run Keithley 2400 [0, 3, 0.01],\
arrKLVMeasMulti =["keith", [
                            [0, 3, 0.01],\
                            [0, -3, -0.01]]]
running = Event()
def exitLoop():
    global running
    global root
    print ("Stop pressed")
    running = False
def start_measurement_thread():
    global running
    running = True
    measurement_thread = threading.Thread(target=fncMeasureMultiple, args=(arrKLVMeasMulti,sample, 1, folderPath))
    measurement_thread.start()
root = tk.Tk()
root.title("Measurements")
start_button = tk.Button(root, text="Start", command=lambda: fncMeasureMultiple(arrKLVMeasMulti,sample, 1, folderPath))
start_button.pack()
exit_button = tk.Button(root, text="Stop", command=lambda: exitLoop)
exit_button.pack()

root.mainloop()
#fncMeasureMultiple(arrKLVMeasMulti,sample, 1, folderPath)

231101_Fab231026_Dev_ShadowMask\Dev08\S63\Sweep1S63Star0_Stop3_Step0.01.csv
231101_Fab231026_Dev_ShadowMask\Dev08\S63\Sweep2S63Star0_StopM3_StepM0.01.csv


In [12]:
#run Keithley 2400 [0, 3, 0.01],\
arrKLVMeasMulti =["keith",[[0, 3, 0.01],\
                            #[0, 3, 0.01],\
                            [0, -3, -0.01]]]
fncMeasureMultiple(arrKLVMeasMulti,sample, 1, folderPath)

240805_Fab240727_NiNiO_NegLOnBrassSM\Dev07\E02\Sweep1E02Star0_Stop3_Step0.01.csv
240805_Fab240727_NiNiO_NegLOnBrassSM\Dev07\E02\Sweep2E02Star0_StopM3_StepM0.01.csv


TclError: can't invoke "destroy" command: application has been destroyed

In [31]:
#run picoamp meter

arrPicoVMeasMulti =["pico", [[0, 1.5, 0.01],\
                [0, -1.5, -0.01],\
                [0, -500, -1],\
                 [0, 1.5, 0.01],\
                 [0, -1.5, -0.01]]]
# Create a thread for the loop
# Create a button to start the loop

# loopThread = threading.Thread(target=fncMeasureMultiple(arrPicoVMeasMulti,sample, 3, folderPath))

# # Start the thread
# loopThread.start()
# root.mainloop()
#fncMeasureMultiple(arrPicoVMeasMulti,sample, 3, folderPath)

231101_Fab231026_Dev_ShadowMask\Dev08\S42\Sweep3S42Star0_Stop1.5_Step0.01.csv
231101_Fab231026_Dev_ShadowMask\Dev08\S42\Sweep4S42Star0_StopM1.5_StepM0.01.csv
231101_Fab231026_Dev_ShadowMask\Dev08\S42\Sweep5S42Star0_StopM500_StepM1.csv
231101_Fab231026_Dev_ShadowMask\Dev08\S42\Sweep6S42Star0_Stop1.5_Step0.01.csv
231101_Fab231026_Dev_ShadowMask\Dev08\S42\Sweep7S42Star0_StopM1.5_StepM0.01.csv


In [16]:
#run Keithley 2400
arrKLVMeasMulti =["keith", [[0, 3, 0.01],\
                [0, -3, -0.01]]]
# loopThread = threading.Thread(target=fncMeasureMultiple(arrKLVMeasMulti,sample, 8, folderPath))
# loopThread.start()
start_button = tk.Button(root, text="Start", command=lambda: fncMeasureMultiple(arrKLVMeasMulti,sample, 8, folderPath))
start_button.pack()
exit_button = tk.Button(root, text="Stop", command=lambda: exitLoop)
        exit_button.pack()
root.mainloop()
#fncMeasureMultiple(arrKLVMeasMulti,sample, 8, folderPath)

231101_Fab231026_Dev_ShadowMask\Dev08\S42\Sweep8S42Star0_Stop3_Step0.01.csv
231101_Fab231026_Dev_ShadowMask\Dev08\S42\Sweep9S42Star0_StopM3_StepM0.01.csv


TclError: can't invoke "button" command: application has been destroyed

In [27]:
arrVMeas = [0, -3, -0.01]
fncMeasureOne(arrVMeas, sample, 1, folderPath)

Incorrect device name


In [45]:
#run picoamp meter
arrPicoVMeasMulti =["keith", [[0, 3, 0.01]]]#,\
                #[0, -1.5, -0.01],\
                #[0, -500, -1],\
                 #[0, 1.5, 0.01],\
                 #[0, -1.5, -0.01]]]
fncMeasureMultiple(arrPicoVMeasMulti,sample, 8, folderPath)

231101_Fab231026_Dev_ShadowMask\Dev06\S45\Sweep8S45Star0_Stop3_Step0.01.csv
